In [ ]:
import os
from backoff import on_exception, expo
import json
import requests
from collections import Counter
from datetime import datetime
from ratelimit import limits, sleep_and_retry, RateLimitException
import time
import urllib.request

In [ ]:
data_path = "D:\\METIS\\data\\"
image_path = "D:\\METIS\\images\\"

In [ ]:
with open(data_path + 'urls.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
data[0]

In [ ]:
@sleep_and_retry
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=60, period=1)
def get_met_response(link, return_page=True):     
    response = requests.get(link)
    page = response.text
    responses.append(page)
    
    if return_page == True:
        return page

In [ ]:
def get_img(link, image_path):
    object_id = link.split('/')[-1]
    filename = image_path + object_id + ".jpg"
    
    im = urllib.request.urlretrieve(link, filename)

In [ ]:
links = []

for obj in data:
    links.append(obj['api_url'])

In [ ]:
failed = []
successful = []

In [ ]:
start = datetime.now()
# just based on previous scrape
for count, link in enumerate(links[4863:]):
    
    obj_id = link.split('/')[-1]
    
    if count % 100 == 0:
            time.sleep(5)
            
    if count % 1000 == 0:
        elapsed = datetime.now() - start
        print("{}: {} | elapsed: {}".format(count, time.ctime(),str(elapsed)))
        
    try:
        response = get_met_response(link, return_page=True)
        img_url = json.loads(response)['primaryImage']
        get_img(img_url, image_path)
        
        successful.append(obj_id)
        
        with open(data_path + "met_responses.json", "w") as write_file:
            json.dump(successful, write_file)
    except:
        failed.append(obj_id)
        
        with open(data_path + "failed_responses.json", "w") as write_file:
            json.dump(failed, write_file)